# Evaluate the box below to initialize the web UI.

In [ ]:
import os
import pprint
import ipywidgets as widgets
import ray

from IPython.display import display

ray.init(redis_address=os.environ["REDIS_ADDRESS"])

**Evaluate the box below to search for objects.**

In [ ]:
object_search = widgets.Text(
    value="",
    placeholder="Object ID",
    description="Search for an object:",
    disabled=False
)
display(object_search)

def handle_submit(sender):
    pp = pprint.PrettyPrinter()
    pp.pprint(ray.global_state.object_table(object_search.value))

object_search.on_submit(handle_submit)

**Evaluate the box below to search for tasks.**

In [ ]:
task_search = widgets.Text(
    value="",
    placeholder="Task ID",
    description="Search for a task:",
    disabled=False
)
display(task_search)

def handle_submit(sender):
    pp = pprint.PrettyPrinter()
    pp.pprint(ray.global_state.task_table(task_search.value))

task_search.on_submit(handle_submit)